In [ ]:
!wget https://mslearntensorflowlp.blob.core.windows.net/data/oxpets_images.tar.gz
!tar xfz oxpets_images.tar.gz
!rm oxpets_images.tar.gz

In [1]:
import os, warnings
import matplotlib.pyplot as plt

import torch
from glob import glob
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, SubsetRandomSampler
from torch.utils.data.dataset import Dataset
from torchvision import transforms
import torch.optim as optim
from torch import nn
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

In [2]:
def Freeze(model_params):
    for param in model_params:
        param.requires_grad = False
        
def UnFreeze(model_params):
    for param in model_params:
        param.requires_grad = True

def Load(filename) :
    img = Image.open(filename)
    img = img.convert('RGB')
    return img

Загружаем обе модели

In [3]:
from torchvision.models.resnet import resnet34 as resnet34
resnet = resnet34(pretrained = True)

In [ ]:
from torchvision.models.vgg import vgg16 as vgg16
vgg16 = vgg16(pretrained = True)

Обрабатываем файлы

Опишем класс датасета и класс для преобразования датаестов

In [ ]:
img_list=glob('images/*.jpg')
class Pet_dataset(Dataset):
    
    def __init__(self, data, transforms=None):
        self.data = data
        self.len = len(data)
        self.transforms = transforms
    
    def __getitem__(self, index):
        img, label = self.data[index]
        
        if self.transforms:
            img = self.transforms(img)
            
        return img, label
    
    def __len__(self):
        return self.len

class Transform_dataset():
    
    def __init__(self, data, num_cl, val_split=0.2, train_transforms=None, val_transforms=None):
        class_values = [[] for x in range(num_cl)]
        
        for d in data:
            class_values[d[1].item()].append(d)
            
        self.train_data = []
        self.val_data = []
        
        for class_dp in class_values:
            split_idx = int(len(class_dp)*(1-val_split))
            self.train_data += class_dp[:split_idx]
            self.val_data += class_dp[split_idx:]
            
        self.train_ds = Pet_dataset(self.train_data, transforms=train_transforms)
        self.val_ds = Pet_dataset(self.val_data, transforms=val_transforms)

In [ ]:
classes = set()

data = []
labels = []

for image in img_list:
    class_name = image.rsplit('_', 1)[0]
    classes.add(class_name)
    img = Load(image)

    data.append(img)
    labels.append(class_name)

class2idx = {cl: idx for idx, cl in enumerate(classes)}        
labels = torch.Tensor(list(map(lambda x: class2idx[x], labels))).long()

data = list(zip(data, labels))

Опишем алгоритмы преобразования тензоров, полученных из файлов. В обучающей выборке производим аугментацию изображений, схожую с предыдущей частью ЛР

In [ ]:
train_transforms = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(45),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_transforms =transforms.Compose([
        transforms.Resize((224,224)),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
 ])

Td = Transform_dataset(data, len(classes), val_split=0.2, train_transforms=train_transforms, val_transforms=val_transforms)

In [ ]:
bn_types = (nn.BatchNorm1d, nn.BatchNorm2d, nn.BatchNorm3d)

class AdaptiveConcatPool2d(nn.Module):
    def __init__(self, sz = None):# конкатенация слоев
        super(AdaptiveConcatPool2d, self).__init__()
        self.output_size = sz or 1
        self.ap = nn.AdaptiveAvgPool2d(self.output_size)
        self.mp = nn.AdaptiveMaxPool2d(self.output_size)

    def forward(self, x): return torch.cat([self.mp(x), self.ap(x)], 1)

def head_blocks(in_dim, p, out_dim, activation=None):# линейный блок
    layers = [
        nn.BatchNorm1d(in_dim),
        nn.Dropout(p),
        nn.Linear(in_dim, out_dim)
    ]
    
    if activation is not None:
        layers.append(activation)
        
    return layers       
    
def create_head(nf, nc, bn_final=False): # приделываем голову к ResNet
    pool = AdaptiveConcatPool2d()
    layers = [pool, nn.Flatten()]
    layers += head_blocks(nf, 0.25, 512, nn.ReLU(inplace=True))
    layers += head_blocks(512, 0.5, nc)
    
    if bn_final:
        layers.append(nn.BatchNorm1d(nc, momentum=0.01))
    
    return nn.Sequential(*layers)
    
def requires_grad(layer): 
    ps = list(layer.parameters())
    if not ps: return None
    return ps[0].requires_grad

def cnn_model(model, nc, bn_final=False, init=nn.init.kaiming_normal_): # Собираем Франкенштейна
    
    body = nn.Sequential(*list(model.children())[:-2])
    head = create_head(1024, nc, bn_final)
    
    model = nn.Sequential(body, head)
    
    Freeze(model[0].parameters())
    
    for child in model[1].children():
        if isinstance(child, nn.Module) and (not isinstance(child, bn_types)) and requires_grad(child): 
            init(child.weight)
    
    return model

num_classes = len(classes)
model_resnet34 = cnn_model(resnet, num_classes, bn_final=True)
model_vgg16 = cnn_model(vgg16, num_classes, bn_final=True)

В качестве оптимизаторов будем использовать алгоритм Адам с lr=10e-3, исходя из результата оптимизации параметров предыдущей части ЛР

In [ ]:
train_indices = list(range(len(Td.train_ds)))
test_indices = list(range(len(Td.val_ds)))


train_loader = DataLoader(Td.train_ds, batch_size=64, shuffle=True)
test_loader = DataLoader(Td.val_ds, batch_size=64, shuffle=True)
criterion = nn.CrossEntropyLoss()
optimizer_resnet = optim.Adam(model_resnet34.parameters(), lr=0.001)
optimizer_vgg = optim.Adam(model_vgg16.parameters(), lr=0.001)

Обучаем и смотрим на результаты ResNet34

In [ ]:
device = 'cuda:0'
model_resnet34.to(device)

def train(epochs, optimizer, model):
    for epoch in range(epochs):
        running_loss = 0
        n_correct = 0
        
        model.train()
        for batch in train_loader:
            inputs, labels = batch
            
            inputs = inputs.to(device)
            labels = labels.to(device)
    
            optimizer.zero_grad()
            
            outputs = model(inputs)
            
            loss = criterion(outputs, labels)
            loss.backward()
            
            optimizer.step()
            
            _, predicted = torch.max(outputs, 1)
            n_correct += (predicted == labels).sum().item()
            
            running_loss += loss.item()
        
        train_acc = 100. * n_correct / len(Td.train_ds)
        train_loss = running_loss / len(train_loader)
        
        n_val_correct = 0
        val_loss = 0
  
        model.eval()
        with torch.no_grad():
            for batch in test_loader:
                inputs, labels = batch
                
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                outputs = model(inputs)
                val_loss = criterion(outputs, labels).item()

                n_val_correct += (torch.max(outputs.data, 1)[1] == labels).sum().item()
                
                
                
        val_acc = 100. * n_val_correct / len(Td.val_ds)
                
                
                
            
        print('Epoch %s: Train Accuracy: %.2f percent, Validation Accuracy: %.2f percent, Train Loss: %s, Validation Loss: %s' 
              % (epoch, train_acc, val_acc, train_loss, val_loss))
        
train(5, optimizer_resnet, model_resnet34)

Обучаем и смотрим на результаты VGG16

In [ ]:
device = 'cuda:0'
model_vgg16.to(device)
        
train(5, optimizer_vgg, model_vgg16)

Несмотря на то, что VGG16 чуть менее точная на обучающей выборке, на валидационной у неё точность выше, а также сильно ниже цена ошибки на ваидационной же выборке, поэтому выбираем её.


In [ ]:
def binary_class(val_labels, pred):  # бинарная классификация
    v, r = [], []
    for i in val_labels:
        if i < 12:
            v.append(0)
        else:
            v.append(1)
    for i in pred:
        if i < 12:
            r.append(0)
        else:
            r.append(1)
    return v, r


In [ ]:
predict = []  # собираем полную валидационную выборку
labels = []
for i in test_loader:
    input, label = i
    input = input.to(device)
    label = label.to(device)
    y_pred = model_vgg16(input)
    _, pred = torch.max(y_pred, 1)
    for j in range(len(pred)):
        predict.append(pred[j].item())
        labels.append(label[j].item())

pred, lab = binary_class(labels, predict)


Теперь посмотрим на точность при бинарной классификации

In [ ]:
sum_bin = 0
for i in range(len(predict)):
  if pred[i] == lab[i]:
    sum_bin+=1
print("Binary accuracy:", sum_bin/len(predict))


Точность 95%, очень хороший показатель

Посмотрим на ConfusionMatrix

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
ConfusionMatrixDisplay.from_predictions(lab, pred)
plt.show()

Как мы видим - точность бинарной классификации на высоте. FP и FN одинаково мало, это значит, что сеть хорошо разделяет, как кошек, так и собак. Видим, что собак в датасете больше.

Определим функцию, которая возвращает n наиболее вероятных предсказанных лейблов в порядке убывания

In [ ]:
def get_data():
    pred1 = []
    pred2 = []
    predict = []
    label1 = []
    l2 = []
    label3 = []
    for i in test_loader:
        input, label = i
        input = input.to(device)
        label = label.to(device)
        y_pred = model_resnet34(input)
        pred1.append(y_pred)
        label1.append(label)
    for i in pred1:
        for j in i:
            for k in j:
                pred2.append(k.item())
            predict.append(pred2)
            pred2 = []
    predict = np.array(predict)
    for i in label1:
        for j in i:
            label3.append(j.item())
    label3 = np.array(label3)
    return predict, label3


def top_acc(valid, n):  # считает лейбл наиболее вероятного лейбла
    res = np.empty((0, n))
    for i in valid:
        ind = np.flip(i.argsort())[:n]
        res = np.append(res, [ind], axis=0)
    return res


def topn_acc(val_labels, pred):  # считает top-n accuracy
    sum_corr = 0
    for (i, elem) in enumerate(val_labels):
        if elem in pred[i]:
            sum_corr += 1
    res = sum_corr / val_labels.shape[0]
    return res


In [ ]:
predict, label3 = get_data()

Подсчитаем top-3 и top-5 accuracy


In [ ]:
y3 =  top_acc(predict,3)
y5 = top_acc(predict,5)

In [ ]:
print('top-3 accuracy:', topn_acc(label3,y3))

In [ ]:
print('top-5 accuracy:', topn_acc(label3,y5))

Как мы видим top-3 accuracy значительно превосходит top-1 accuracy(aka посчитанная точность), а вот уже различия между top-3 accuracy и top-5 accuracy очень мало.

In [38]:
print('top-3 accuracy:', topn_acc(label3,y3))

top-3 accuracy: 0.9830966869506423


In [39]:
print('top-5 accuracy:', topn_acc(label3,y5))

top-5 accuracy: 0.9891818796484111


Как мы видим top-3 accuracy значительно превосходит top-1 accuracy(aka посчитанная точность), а вот уже различия между top-3 accuracy и top-5 accuracy очень мало.